In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/home/beans/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import *

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'e27f0tqo'

In [3]:
wandb.init(id='1rfauwvw', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

14029.957

In [5]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [6]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=5e-5)

In [7]:
%%time
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

CPU times: user 26.2 s, sys: 806 ms, total: 27 s
Wall time: 27 s


In [8]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


In [9]:
set_lr(42)

In [9]:
model_stem = "11.16"
START_E = 105

In [10]:
n_epochs = 10000
log_wandb = True

for e in range(START_E, n_epochs):
    
    lr = get_lr()
    opt.param_groups[0]['lr'] = lr if lr!=42 else (6e-4 if e < 20 else 3e-4 if e < 100 else 1e-4)

    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2560)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 105

{'trn_control_loss': 0.01883623, 'trn_headings_loss': 0.00093575, 'trn_curvatures_loss': 0.00353149, 'consistency losses/trn_steer_cost': 0.00651704, 'consistency losses/trn_te_loss': 23.10754675, 'aux losses/trn_pitch_loss': 1.37120438, 'aux losses/trn_yaw_loss': 0.38169289, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.96875, 'consistency losses/trn_torque_loss': 3104.3736515, 'consistency losses/trn_torque_delta_loss': 26.35386794, 'logistical/obs_consumed_per_second': 192.05511811, 'logistical/obs_generated_per_second': 168.23622047, 'logistical/slowest_runner_obs_per_sec': 8.77165354, 'logistical/data_consumption_ratio': 1.14548004, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.20781136, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 189.9453125, 'logistical/obs_generated_per_second': 169.390625, 'logistical/slowest_runner_obs_per_sec': 9.515625, 'logistical/data_consumption_ratio': 1.12168805, 'logistical/manual_train_pau

{'logistical/obs_consumed_per_second': 195.9765625, 'logistical/obs_generated_per_second': 186.8984375, 'logistical/slowest_runner_obs_per_sec': 10.5234375, 'logistical/data_consumption_ratio': 1.05636431, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01863405, 'trn_headings_loss': 0.00096844, 'trn_curvatures_loss': 0.00350057, 'consistency losses/trn_steer_cost': 0.00522856, 'consistency losses/trn_te_loss': 24.45264418, 'aux losses/trn_pitch_loss': 1.42435074, 'aux losses/trn_yaw_loss': 0.29352283, 'consistency losses/trn_%_updates_w_torque_delta_loss': 1.0, 'consistency losses/trn_torque_loss': 2881.78493023, 'consistency losses/trn_torque_delta_loss': 32.26791798, 'logistical/max_param': 16.21724129, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 199.8984375, 'logistical/obs_generated_per_second': 204.0703125, 'logistical/slowest_runner_obs_per_sec': 13.0, 'logistical/data_consumption_ratio': 0.98034863, 'logistical/manual_train_pause': 0.0, 'trn_cont

{'logistical/obs_consumed_per_second': 194.0546875, 'logistical/obs_generated_per_second': 182.734375, 'logistical/slowest_runner_obs_per_sec': 1.0, 'logistical/data_consumption_ratio': 1.06322935, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02155298, 'trn_headings_loss': 0.00105032, 'trn_curvatures_loss': 0.0042659, 'consistency losses/trn_steer_cost': 0.00740351, 'consistency losses/trn_te_loss': 22.18435507, 'aux losses/trn_pitch_loss': 1.43882751, 'aux losses/trn_yaw_loss': 0.3302989, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3316.04710388, 'consistency losses/trn_torque_delta_loss': 32.5311332, 'logistical/max_param': 16.22005081, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 190.2265625, 'logistical/obs_generated_per_second': 179.4609375, 'logistical/slowest_runner_obs_per_sec': 1.0, 'logistical/data_consumption_ratio': 1.06011666, 'logistical/manual_train_pause': 0.0, 'trn_control_lo

{'logistical/obs_consumed_per_second': 198.1015625, 'logistical/obs_generated_per_second': 187.015625, 'logistical/slowest_runner_obs_per_sec': 9.6796875, 'logistical/data_consumption_ratio': 1.06046628, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.02607021, 'trn_headings_loss': 0.00112149, 'trn_curvatures_loss': 0.00442763, 'consistency losses/trn_steer_cost': 0.00785856, 'consistency losses/trn_te_loss': 22.76331806, 'aux losses/trn_pitch_loss': 1.42242432, 'aux losses/trn_yaw_loss': 0.32288504, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3373.0900383, 'consistency losses/trn_torque_delta_loss': 35.40008023, 'logistical/max_param': 16.22323418, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 197.359375, 'logistical/obs_generated_per_second': 187.1484375, 'logistical/slowest_runner_obs_per_sec': 10.0, 'logistical/data_consumption_ratio': 1.05750968, 'logistical/manual_train_pause': 0.0, 'trn_co

{'logistical/obs_consumed_per_second': 194.4765625, 'logistical/obs_generated_per_second': 186.4765625, 'logistical/slowest_runner_obs_per_sec': 10.4296875, 'logistical/data_consumption_ratio': 1.04942381, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01883245, 'trn_headings_loss': 0.00100747, 'trn_curvatures_loss': 0.00306185, 'consistency losses/trn_steer_cost': 0.00567093, 'consistency losses/trn_te_loss': 23.46693073, 'aux losses/trn_pitch_loss': 1.4126358, 'aux losses/trn_yaw_loss': 0.32665062, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9765625, 'consistency losses/trn_torque_loss': 3305.08122253, 'consistency losses/trn_torque_delta_loss': 37.58523945, 'logistical/max_param': 16.24120331, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 194.859375, 'logistical/obs_generated_per_second': 184.984375, 'logistical/slowest_runner_obs_per_sec': 10.9765625, 'logistical/data_consumption_ratio': 1.05597797, 'logistical/manual_train_pause': 0.0, 

...

{'logistical/obs_consumed_per_second': 190.046875, 'logistical/obs_generated_per_second': 190.640625, 'logistical/slowest_runner_obs_per_sec': 11.921875, 'logistical/data_consumption_ratio': 0.99753232, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01432663, 'trn_headings_loss': 0.00085149, 'trn_curvatures_loss': 0.00273747, 'consistency losses/trn_steer_cost': 0.00432997, 'consistency losses/trn_te_loss': 24.79600972, 'aux losses/trn_pitch_loss': 1.36698914, 'aux losses/trn_yaw_loss': 0.27415705, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.984375, 'consistency losses/trn_torque_loss': 3495.68680573, 'consistency losses/trn_torque_delta_loss': 34.06216033, 'logistical/max_param': 16.2481575, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 195.6171875, 'logistical/obs_generated_per_second': 198.375, 'logistical/slowest_runner_obs_per_sec': 13.546875, 'logistical/data_consumption_ratio': 0.98807587, 'logistical/manual_train_pause': 0.0, 'trn_co

{'logistical/obs_consumed_per_second': 195.0859375, 'logistical/obs_generated_per_second': 182.890625, 'logistical/slowest_runner_obs_per_sec': 7.09375, 'logistical/data_consumption_ratio': 1.07014291, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01229188, 'trn_headings_loss': 0.00064588, 'trn_curvatures_loss': 0.00215371, 'consistency losses/trn_steer_cost': 0.00466773, 'consistency losses/trn_te_loss': 23.50298287, 'aux losses/trn_pitch_loss': 1.23970795, 'aux losses/trn_yaw_loss': 0.26877165, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3112.11986446, 'consistency losses/trn_torque_delta_loss': 30.72913631, 'logistical/max_param': 16.24767113, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 199.7109375, 'logistical/obs_generated_per_second': 189.65625, 'logistical/slowest_runner_obs_per_sec': 12.796875, 'logistical/data_consumption_ratio': 1.05577735, 'logistical/manual_train_pause': 0.0, 'trn

{'logistical/obs_consumed_per_second': 196.921875, 'logistical/obs_generated_per_second': 174.5390625, 'logistical/slowest_runner_obs_per_sec': 1.6640625, 'logistical/data_consumption_ratio': 1.13548026, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01921174, 'trn_headings_loss': 0.00102187, 'trn_curvatures_loss': 0.00398188, 'consistency losses/trn_steer_cost': 0.00518614, 'consistency losses/trn_te_loss': 23.72464232, 'aux losses/trn_pitch_loss': 1.17047119, 'aux losses/trn_yaw_loss': 0.26299047, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3406.87082005, 'consistency losses/trn_torque_delta_loss': 35.27812282, 'logistical/max_param': 16.23559761, 'logistical/lr': 0.0003}
{'logistical/obs_consumed_per_second': 191.859375, 'logistical/obs_generated_per_second': 166.8828125, 'logistical/slowest_runner_obs_per_sec': 1.0, 'logistical/data_consumption_ratio': 1.15153637, 'logistical/manual_train_pause': 0.0, 'trn_co

{'logistical/obs_consumed_per_second': 189.8203125, 'logistical/obs_generated_per_second': 175.6953125, 'logistical/slowest_runner_obs_per_sec': 1.0, 'logistical/data_consumption_ratio': 1.08206741, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.01400745, 'trn_headings_loss': 0.00080952, 'trn_curvatures_loss': 0.0026014, 'consistency losses/trn_steer_cost': 0.0040324, 'consistency losses/trn_te_loss': 24.54784907, 'aux losses/trn_pitch_loss': 1.3083725, 'aux losses/trn_yaw_loss': 0.27241087, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.9921875, 'consistency losses/trn_torque_loss': 3505.06873798, 'consistency losses/trn_torque_delta_loss': 33.85540385, 'logistical/max_param': 16.22058105, 'logistical/lr': 0.0003}


KeyboardInterrupt: 

In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e63.torch")

In [15]:
set_should_stop(True)

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s
